In [1]:
%pip install huggingface_hub --quiet
%pip install transformers --quiet
%pip install torch torchvision torchaudio --quiet
%pip install --upgrade pip --quiet
%pip install tensorflow --quiet
%pip install python-dotenv --quiet
%pip install --upgrade --upgrade-strategy eager "optimum[openvino]" --quiet
%pip install tf-keras --quiet
%pip install sentence-transformers --quiet
%pip install langchain_community --quiet
%pip install langchain_openai --quiet
%pip install pypdf --quiet
%pip install chromadb --quiet
%pip install langchain-chroma --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import shutil
import logging
import os
import gc
from pathlib import Path
from dotenv import load_dotenv

from huggingface_hub import login, whoami
from optimum.intel import OVQuantizer
from optimum.intel.openvino import OVModelForCausalLM
import openvino as ov
import nncf

nncf.set_log_level(logging.ERROR)

load_dotenv(verbose=True)
cache_dir = os.environ['CACHE_DIR']

/home/hpcuser/joycehcl/openvino/openvino_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-29 07:58:01.898538: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 07:58:01.914407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730188681.933902 3577971 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730188681.939446 3577971 cuda

In [3]:
def prepare_model(model_vendor, model_id, group_size:int, ratio:float, int4_mode:str='SYM', generate_fp16:bool=True, generate_int8:bool=True, generate_int4:bool=True, cache_dir='./cache'):
    pt_model_id = f'{model_vendor}/{model_id}'
    fp16_model_dir = Path(model_id) / "FP16"
    int8_model_dir = Path(model_id) / "INT8"
    int4_model_dir = Path(model_id) / "INT4"

    ov_model_file_name = 'openvino_model.xml'

    print(f'** Prepaing model : {model_vendor}/{model_id}')

    # FP16
    if generate_fp16 and not os.path.exists(fp16_model_dir / ov_model_file_name):
        print('\n** Generating an FP16 IR model')
        ov_model = OVModelForCausalLM.from_pretrained(pt_model_id, export=True, compile=False, cache_dir=cache_dir, ov_config={'CACHE_DIR':cache_dir})
        ov_model.half()
        ov_model.save_pretrained(fp16_model_dir)
        del ov_model
        gc.collect()
    else:
        print('\n** Skip generation of FP16 IR model (directory already exists)')

    # INT8
    if generate_int8 and not os.path.exists(int8_model_dir / ov_model_file_name):
        print('\n** Generating an INT8 IR model')
        ov_model = OVModelForCausalLM.from_pretrained(fp16_model_dir, compile=False, cache_dir=cache_dir, ov_config={'CACHE_DIR':cache_dir})
        quantizer = OVQuantizer.from_pretrained(ov_model, cache_dir=cache_dir)
        quantizer.quantize(save_directory=int8_model_dir, weights_only=True)
        del quantizer
        del ov_model
        gc.collect()
    else:
        print('\n** Skip generation of INT8 IR model (directory already exists)')

    # INT4
    if generate_int4 and not os.path.exists(int4_model_dir / ov_model_file_name):
        print(f'\n** Generating an INT4_{int4_mode} IR model')
        ov_model = OVModelForCausalLM.from_pretrained(fp16_model_dir, compile=False, cache_dir=cache_dir, ov_config={'CACHE_DIR':cache_dir})
        int4_model_dir.mkdir(parents=True, exist_ok=True)
        ov_model = ov.Core().read_model(fp16_model_dir / ov_model_file_name)
        shutil.copy(fp16_model_dir / 'config.json', int4_model_dir / 'config.json')
        comp_mode = nncf.CompressWeightsMode.INT4_ASYM if int4_mode=='ASYM' else nncf.CompressWeightsMode.INT4_SYM
        compressed_model = nncf.compress_weights(ov_model, mode=comp_mode, ratio=ratio, group_size=group_size)
        ov.save_model(compressed_model, int4_model_dir / ov_model_file_name)
        del ov_model
        del compressed_model
        gc.collect()
    else:
        print('\n** Skip generation of INT4 IR model (directory already exists)')

In [ ]:
# hf_token = hf_xxxxxx

In [4]:
print('*** LLM model downloader')
try:
    whoami()
    print('Authorization token already provided')
except OSError:
    print('The llama2 model is a controlled model.')
    print('You need to login to HuggingFace hub to download the model.')
    login()
finally:
    prepare_model('meta-llama', 'Llama-3.2-1B', group_size=128, ratio=0.8)

*** LLM model downloader
Authorization token already provided
** Prepaing model : meta-llama/Llama-3.2-1B

** Generating an FP16 IR model


We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
/home/hpcuser/joycehcl/openvino/openvino_env/lib/python3.10/site-packages/transformers/cache_utils.py:447: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/home/hpcuser/joycehcl/openvino/openvino_env/lib/python3.10/site-packages/optimum/exporters/openvino/model_patcher.py:496: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace

/home/hpcuser/joycehcl/openvino/openvino_env/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


** Generating an INT8 IR model


`quantization_config` was not provided. In the future, please provide `quantization_config`
Calibration dataset was not provided, assuming weight only quantization.



** Generating an INT4_SYM IR model


In [5]:
from langchain.document_loaders import  DirectoryLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
import os
import shutil

os.environ["OPENAI_API_BASE"] ='http://10.35.151.101:8001/v1'
os.environ["OPENAI_API_KEY"] = "sk-1234"
CHROMA_PATH = "docs_embedding"

In [6]:
def load_documents():
  document_loader = PyPDFDirectoryLoader('docs') 
  return document_loader.load()

In [7]:
def split_text(documents: list[Document]):

  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # Size of each chunk in characters
    chunk_overlap=100, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
  )

  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
  document = chunks[0]
  print(document.page_content)
  print(document.metadata)

  return chunks

In [8]:
def save_to_chroma(chunks: list[Document]):
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)
    db = Chroma.from_documents(
      chunks,
      OpenAIEmbeddings(),
      persist_directory=CHROMA_PATH
    )

    db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [9]:
def generate_data_store():
  documents = load_documents()
  chunks = split_text(documents)
  save_to_chroma(chunks)

print('*** Converting documents into embeddings and creating a vector store(s)')
generate_data_store()

*** Converting documents into embeddings and creating a vector store(s)
Split 34 documents into 173 chunks.
(Affiliated to C.I.S.C.E, New Delhi)  
 
 
 
 
 
 
 
 
 
 
 
 
HR POLICIES 
Handbook
{'source': 'docs/NHPS-POLICIES-Revised-1.pdf', 'page': 0, 'start_index': 12}
Saved 173 chunks to docs_embedding.


In [60]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

/home/hpcuser/joycehcl/openvino/openvino_env/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


INT 4

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_chroma import Chroma


vectorstore_dir   = os.environ['VECTOR_DB_DIR']
embeddings_model  = os.environ['MODEL_EMBEDDINGS']


embeddings = OpenAIEmbeddings()
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""


vectorstore_dir = f'{vectorstore_dir}'
vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
retriever = vectorstore.as_retriever()
#    search_type='similarity_score_threshold', 
#    search_kwargs={
#        'score_threshold' : 0.8, 
#        'k' : 4
#    }
#)

# results = vectorstore.similarity_search_with_relevance_scores(query_text, k=3)

# if len(results) == 0 or results[0][1] < 0.7:
#    print(f"Unable to find matching results.")

# context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

# prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# prompt = prompt_template.format(context=context_text, question=query_text)
print(f'** Vector store : {vectorstore_dir}')

** Vector store : docs_embedding


In [12]:
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA


model_id = 'Llama-3.2-1B'
model_name        = os.environ['MODEL_NAME']
model_precision   = os.environ['MODEL_PRECISION']
inference_device  = os.environ['INFERENCE_DEVICE']
ov_config         = {"PERFORMANCE_HINT":"LATENCY", "NUM_STREAMS":"1", "CACHE_DIR":cache_dir}
num_max_tokens    = int(os.environ['NUM_MAX_TOKENS'])
rag_chain_type    = os.environ['RAG_CHAIN_TYPE']

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
ov_model_path = f'./{model_name}/{model_precision}'
model = OVModelForCausalLM.from_pretrained(model_id=ov_model_path, device=inference_device, ov_config=ov_config, cache_dir=cache_dir)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=num_max_tokens)
llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type=rag_chain_type, retriever=retriever)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/tmp/ipykernel_3577971/556153035.py:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [13]:
text_user_en = "Who administrates the Promotions?"
ans = qa_chain.run(text_user_en)
print(ans)

/tmp/ipykernel_3577971/4141815617.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ans = qa_chain.run(text_user_en)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: Who administrates the Promotions?
Helpful Answer: The Promotions department is responsible for the promotion of the company's products and services. They are responsible for creating and implementing marketing strategies, advertising campaigns, and promotional activities to increase brand awareness and sales. They also work closely with the sales team to ensure that the products are properly marketed and promoted to the target audience. The Promotions department is responsible for managing the company's promotional budget, ensuring that the budget is spent effectively and efficiently, and that the company's promotional activities are aligned with the company's overall marketing strategy. The Promotions department is also responsible for managing the company's social media accounts, creating and managing promotion

In [36]:
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA


model_id = 'Llama-3.2-1B'
model_name        = os.environ['MODEL_NAME']
model_precision   = "INT4"
inference_device  = os.environ['INFERENCE_DEVICE']
ov_config         = {"PERFORMANCE_HINT":"LATENCY", "NUM_STREAMS":"1", "CACHE_DIR":cache_dir}
num_max_tokens    = 200
rag_chain_type    = os.environ['RAG_CHAIN_TYPE']

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
ov_model_path = f'./{model_name}/{model_precision}'
model = OVModelForCausalLM.from_pretrained(model_id=ov_model_path, device=inference_device, ov_config=ov_config, cache_dir=cache_dir)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=num_max_tokens)
llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type=rag_chain_type, retriever=retriever)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [37]:
text_user_en = "What is the promotion policy for non teaching staff?"
ans = qa_chain.run(text_user_en)
print(ans)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: What is the promotion policy for non teaching staff?
Helpful Answer: The promotion policy for non teaching staff is that they are promoted based on merit. This means that they are promoted based on their performance and qualifications. The promotion policy for teaching staff is that they are promoted based on their performance and qualifications, and also based on seniority. This means that they are promoted based on their length of service and their ability to perform their job well. The promotion policy for non teaching staff is that they are promoted based on merit. This means that they are promoted based on their performance and qualifications. The promotion policy for teaching staff is that they are promoted based on their performance and qualifications, and also based on seniority. This means that they are 

Int 8

In [53]:
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA


model_id = 'Llama-3.2-1B'
model_name        = os.environ['MODEL_NAME']
model_precision   = "INT8"
inference_device  = os.environ['INFERENCE_DEVICE']
ov_config         = {"PERFORMANCE_HINT":"LATENCY", "NUM_STREAMS":"1", "CACHE_DIR":cache_dir}
num_max_tokens    = 100
rag_chain_type    = os.environ['RAG_CHAIN_TYPE']

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
ov_model_path = f'./{model_name}/{model_precision}'
model = OVModelForCausalLM.from_pretrained(model_id=ov_model_path, device=inference_device, ov_config=ov_config, cache_dir=cache_dir)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=num_max_tokens)
llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type=rag_chain_type, retriever=retriever)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [54]:

text_user_en = "Can you tell me about the staff referal incentive policy"
ans = qa_chain.run(text_user_en)
print(ans)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: Can you tell me about the staff referal incentive policy
Helpful Answer: It is a policy in which a company gives employees a bonus for referring customers to the company. The bonus is usually a percentage of the customer's purchase amount, and the employee may also receive a small cash bonus. The policy is designed to encourage employees to refer customers to the company, as it can lead to increased sales and profits for the company. The policy is usually implemented by companies that have a high turnover rate, as it can help to retain employees who are not satisfied with their current job.



In [28]:
query_text = "What is the promotion policy for non teaching staff?"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
results = db.similarity_search_with_relevance_scores(query_text, k=3)

if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")
context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

Unable to find matching results.


In [32]:
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate


model_id = 'Llama-3.2-1B'
model_name        = os.environ['MODEL_NAME']
model_precision   = "INT8"
inference_device  = os.environ['INFERENCE_DEVICE']
ov_config         = {"PERFORMANCE_HINT":"LATENCY", "NUM_STREAMS":"1", "CACHE_DIR":cache_dir}
num_max_tokens    = 200
rag_chain_type    = os.environ['RAG_CHAIN_TYPE']
PROMPT_TEMPLATE   = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

# prompt = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know."
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
ov_model_path = f'./{model_name}/{model_precision}'
model = OVModelForCausalLM.from_pretrained(model_id=ov_model_path, device=inference_device, ov_config=ov_config, cache_dir=cache_dir)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=num_max_tokens)
llm = HuggingFacePipeline(pipeline=pipe)
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
chain = prompt_template | llm
# print(chain.invoke({"context": context_text,"question": query_text}))
response_text = llm.predict(prompt)
# qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type=rag_chain_type, retriever=retriever, chain_type_kwargs={"prompt": prompt})

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [24]:

ans = qa_chain.run(text_user_en)
print(ans)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: What is the promotion policy for non teaching staff? 
Context:  
Answer: 



In [33]:
print(response_text)

Human: 
Answer the question based only on the following context:

 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?
 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?

 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?
 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?
 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?
 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?
 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?
 - -
Answer the question based on the above context: What is the promotion policy for non teaching staff?
 - -
Answer the question based on the above context: What is the promotion policy for

FP16

In [61]:
from transformers import AutoTokenizer, pipeline
from transformers import AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA


model_id = 'Llama-3.2-1B'
model_name        = os.environ['MODEL_NAME']
model_precision   = "FP16"
inference_device  = os.environ['INFERENCE_DEVICE']
ov_config         = {"PERFORMANCE_HINT":"LATENCY", "NUM_STREAMS":"1", "CACHE_DIR":cache_dir}
num_max_tokens    = 200
rag_chain_type    = os.environ['RAG_CHAIN_TYPE']

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
ov_model_path = f'./{model_name}/{model_precision}'
model = OVModelForCausalLM.from_pretrained(model_id=ov_model_path, device=inference_device, ov_config=ov_config, cache_dir=cache_dir)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=num_max_tokens)
llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type=rag_chain_type, retriever=retriever, chain_type_kwargs={"prompt": prompt})

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [45]:
text_user_en = "Can you tell me about the Extension of Probation Period"
ans = qa_chain.run(text_user_en)
print(ans)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Question: Can you tell me about the Extension of Probation Period
Helpful Answer: The extension of the probation period is a legal process that allows a person to extend the probationary period of their sentence beyond the original length of their sentence. This can be done for a variety of reasons, such as if the person is not able to comply with the terms of their probation or if they have a change of circumstances that make it necessary to extend their probation. The length of the extension can vary depending on the circumstances, but it is typically limited to a maximum of two years.


In [63]:

text_user_en = "Can you tell me about the staff referal incentive policy"
ans = qa_chain.run(text_user_en)
print(ans)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Can you tell me about the staff referal incentive policy 
Context:  
Answer:  
Staff referal incentive policy is one of the main policies implemented by the company to encourage staff to refer their friends to the company. This policy allows employees to refer their friends to the company and get a percentage of the referral fee as a reward. The referral fee is calculated based on the number of referrals made by the employee. The company has a dedicated team of recruiters who are responsible for reviewing and approving the referrals made by the employees. The company also provides a support system to the employees who refer their friends to the company, such as training and resources to help them succeed. The company also has a transpa

In [64]:
text_user_en = "Can you tell me about the leave policy?"
ans = qa_chain.run(text_user_en)
print(ans)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: Can you tell me about the leave policy? 
Context:  
Answer: The leave policy is designed to provide flexibility for employees to take time off when they need it. The policy allows employees to take up to 12 weeks of unpaid leave each year, with a maximum of 52 weeks in any 12-month period. This includes parental leave, sick leave, and compassionate leave. The policy also provides for a number of other benefits, such as bereavement leave, family leave, and parental bonding leave. Employees are expected to use their leave in a responsible manner and to return to work as soon as possible. If an employee is unable to return to work, they may be eligible for a short-term disability insurance plan.
